In [17]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import numpy as np

init_notebook_mode(connected=True)

StyleGAN training evaluation

In [18]:
with open('slurm-2551786.out', 'r') as f:
    lines = []
    raw_lines = f.readlines()

    for line in raw_lines[302:]:
        line = [item for item in line.rstrip().split(' ') if item]
        if len(line) == 19:
            line.insert(9, '0h')
            line.insert(9, '0d')
        elif len(line) == 20 and 'h' in line[9]:
            line.insert(9, '0d')
        else:
            line.insert(12, '0s')
        #print(' '.join(line))
        lines.append(line)
    del lines[-1]
    
ticks = []
kimgs = []
lods = []
effective_resolutions = []
times = []
secticks = []
seckimgs = []

def parse_time_list(time):
    seconds = int(time[0].translate({ord('d'): None})) * 24 * 60 * 60
    seconds += int(time[1].translate({ord('h'): None})) * 60 * 60
    seconds += int(time[2].translate({ord('m'): None})) * 60
    seconds += int(time[3].translate({ord('s'): None}))
    return seconds

for row in lines:
    ticks.append(int(row[1]))
    kimgs.append(float(row[3]))
    lod = float(row[5])
    lods.append(lod)
    effective_resolutions.append(2**(10 - lod))
    time_list = row[9:13]
    times.append(parse_time_list(time_list))
    secticks.append(float(row[14]))
    seckimgs.append(float(row[16]))
        

In [6]:
kimg_v_time = go.Scatter(
            x=times,
            y=kimgs,
            mode='lines')

kimg_v_effective_resolution = go.Scatter(
            x=effective_resolutions,
            y=kimgs,
            mode='lines')

seckimgs_v_time = go.Scatter(
                x = times,
                y=seckimgs,
                mode='lines')

from plotly import tools

fig = tools.make_subplots(rows=2, cols=2, subplot_titles=('Amount of images processed versus time', 'Amount of images processed versus effective resolution', 'processing time per kimg versus time'))

fig.append_trace(kimg_v_time, 1, 1)
fig.append_trace(kimg_v_effective_resolution, 1, 2)
fig.append_trace(seckimgs_v_time, 2, 1)

fig['layout']['xaxis1'].update(title='Time in seconds')
fig['layout']['xaxis2'].update(title='Effective resolution')
fig['layout']['xaxis3'].update(title='Time in seconds')

fig['layout']['yaxis1'].update(title='k-images')
fig['layout']['yaxis2'].update(title='k-images')
fig['layout']['yaxis3'].update(title='sec/kimg')

fig['layout'].update(title='StyleGAN Image Processing on 4 * GTX 1080 Ti')
fig['layout'].update(showlegend=False)



iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



In [25]:
with open('metrics_gender.out', 'r') as f:
    lines = []
    raw_lines = f.readlines()
    scores = []

    for line in raw_lines[181:]:
        line = [item for item in line.rstrip().split(' ') if item]
        if line == []:
            break
        scores.append(float(line[-1]))
        
kimg_filt = []
for i in range(0, len(kimgs), 10):
    kimg_filt.append(kimgs[i])
    
with open('metrics_finding.out', 'r') as f:
    lines = []
    raw_lines = f.readlines()
    scores2 = []

    for line in raw_lines[181:]:
        line = [item for item in line.rstrip().split(' ') if item]
        if line == []:
            break
        scores2.append(float(line[-1]))
        
with open('slurm-2598441.out', 'r') as f:
    lines = []
    raw_lines = f.readlines()
    scores3 = []

    for line in raw_lines[134:]:
        line = [item for item in line.rstrip().split(' ') if item]
        if line == []:
            break
        scores3.append(float(line[-1]))
        
print(scores)

[533.991, 431.3086, 347.8145, 379.6959, 323.61, 228.2506, 228.6296, 162.0954, 151.9518, 147.1369, 152.9691, 152.3515, 148.2049]


In [26]:
kimg_v_scores1 = go.Scatter(
                x=kimg_filt,
                y=scores,
                mode='lines')

kimg_v_scores2 = go.Scatter(
                x=kimg_filt,
                y=scores2,
                mode='lines')

kimg_v_scores3 = go.Scatter(
                x=kimg_filt,
                y=scores3,
                mode='lines')

layout = go.Layout(
    title='FID score versus resolution (gender labels)',
    xaxis=dict(
        title='1000 images',
    ),
    yaxis=dict(
        title='FID score',
        range=[0, 550]
    )
)

In [33]:
fig = tools.make_subplots(rows=1, cols=3, subplot_titles=('FID score (gender labels)', 'FID score (finding labels)', 'FID score (Style GAN)'))
fig.append_trace(kimg_v_scores1, 1, 1)
fig.append_trace(kimg_v_scores2, 1, 2)
fig.append_trace(kimg_v_scores3, 1, 3)

fig['layout']['xaxis1'].update(title='1000 images')
fig['layout']['xaxis2'].update(title='1000 images')
fig['layout']['xaxis3'].update(title='1000 images')


fig['layout']['yaxis1'].update(title='FID score')
fig['layout']['yaxis1'].update(range=[0,600])
fig['layout']['yaxis2'].update(title='FID score')
fig['layout']['yaxis2'].update(range=[0,600])
fig['layout']['yaxis3'].update(title='FID score')
fig['layout']['yaxis3'].update(range=[0,600],showline=False)

fig['layout'].update(title='Frechet Inception Distances for conditional Progressive GAN and Style GAN')
fig['layout'].update(showlegend=False)

iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]

